# Employment Rate Data

### Exercises in efficient collection and storage

------

December 29, 2017

-----

GOAL: Collect series from API once and store as one part of a SQL database. Store the date of the latest value and when it is more than X days old, start to check the API for a new datapoint. 


-----

To Do List:

 - Set up config file
 - Annotate existing code
 - Add code to retrieve full time series from API
 - Add code to store in SQL database
 - Move above two sections to area that only runs if no database or db empty.
 - Adjust code to retrieve latest datapoints 
 - Add code to save / replace latest datapoints in SQL db
 - Add code to check API only if X days from latest datapoint
 - Add code to generate note in SQL database of date when updated last
 - Annotate code
 - Clean up code
 

In [16]:
# Import python packages
import requests
print('requests v{}'.format(requests.__version__))
import pandas as pd
print('pandas v{}'.format(pd.__version__))
import sqlite3
print('sqlite3 v{}'.format(sqlite3.version))
import json

import config   # Local file config.py with BLS API key

requests v2.14.2
pandas v0.21.1
sqlite3 v2.6.0


In [17]:
# BLS API - access to Bureau of Labor Statistics data
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(config.bls_key)

# Info for sending with post request
headers = {'Content-type': 'application/json'}
param = {"seriesid": ['LNS12300060'],  # BLS series ID
         "startyear": "2016", "endyear": "2017"}

# Request json data from BLS API
p = requests.post('{}{}'.format(url, key), 
                  data=json.dumps(param), 
                  headers=headers).json()

# Print whether API request was successful
print(p['status']) 

# Location within API results of time series and footnotes
r = p['Results']['series'][0]['data']

REQUEST_SUCCEEDED
REQUEST_SUCCEEDED
REQUEST_SUCCEEDED
REQUEST_SUCCEEDED
1 loop, best of 3: 204 ms per loop


In [18]:
# List comprehensions - clean json data and covert the 
# date, value, and footnote into pandas dataframe columns
dates = ['{} {}'.format(i['period'], i['year']) for i in r]
index = pd.to_datetime(dates)
data = {param['seriesid'][0]: [float(i['value']) for i in r],
        'footnotes': [i['footnotes'][0] for i in r]}
# .iloc[::-1] reverses the sorting/order of the dataframe
df = pd.DataFrame(index=index, data=data).iloc[::-1]

In [22]:
# Test to see if SQL database exists


# Connect to SQL database
conn = sqlite3.connect('econ_data.db')

# Save dataframe to database
df.to_sql(param['seriesid'][0], conn, 
          if_exists='replace', index_label='Date')